# Neural Network with Quadratic Data
- y = 8x^2+7x+3
- 4 hidden layers

## 1.Import Required Libraries

In [3]:
!pip3 install visdom

     |████████████████████████████████| 676 kB 5.3 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=01005a26d53054aa5025908b79cc54642bd67a104b92328d58aced5133f5f5e4
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5710 sha256=20ecc4fffde53a1d235c85826db08ee61853ac3c349efd09205662998ce79ac7
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built visdom torchfile


In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable
from visdom import Visdom

In [5]:
# 코렙에서 visdom 사용 방법

# !npm install -g npm to update!
! npm install -g localtunnel
get_ipython().system_raw('/usr/local/bin/python -m pip install visdom')
get_ipython().system_raw('/usr/local/bin/python -m visdom.server -port 6006 >> visdomlog.txt 2>&1 &')
get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')

import time
time.sleep(5)
! cat url.txt

import visdom
time.sleep(5)
vis = visdom.Visdom(port='6006')
print(vis)
time.sleep(3)
vis.text('testing')
! cat visdomlog.txt
# 아래 출력 메세지의 Your url is 부분의 url 클릭함
# 나타난 웹 페이지의 'click to countinue' 버튼 클릭함

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 1.732s
your url is: https://young-newt-92.loca.lt


Setting up a new session...


/usr/local/lib/python3.7/dist-packages/visdom/server.py:39: DeprecationWarning: zmq.eventloop.ioloop is deprecated in pyzmq 17. pyzmq now works with default tornado and asyncio eventloops.
  ioloop.install()  # Needs to happen before any tornado imports!
INFO:root:Application Started
INFO:tornado.access:200 POST /env/main (127.0.0.1) 0.77ms
INFO:tornado.access:101 GET /vis_socket (127.0.0.1) 0.44ms
INFO:root:Opened visdom socket from ip: 127.0.0.1
INFO:tornado.access:200 POST /events (127.0.0.1) 0.85ms


## 2. Data Generation

In [6]:
num_data = 1000
num_epoch = 5000

noise = init.normal(torch.FloatTensor(num_data, 1), std=1)
x = init.uniform(torch.Tensor(num_data, 1), -15, 15)

y = (x**2) + 3
y_noise = y + noise

x.size(), y.size()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  """


(torch.Size([1000, 1]), torch.Size([1000, 1]))

In [7]:
# visualize data with visdom

input_data = torch.cat([x, y_noise], 1)

win = vis.scatter(
    X = input_data,
    opts=dict(
        xtickmin=-15,
        xtickmax=15,
        xtickstep=1,
        ytickmin=0,
        ytickmax=500,
        ytickstep=1,
        markersymbol='dot',
        markersize=5,
        markercolor=np.random.randint(0, 255, num_data),
    ),
)

# Tensor(n,1) != Tensor(n, 1) 버그 있음
vis.line(
    X=x.reshape(x.shape[0]), 
    Y=y.reshape(y.shape[0]), 
    win=win)

'window_3a50ec3ac1333a'

## 3. Model & Optimizer

In [8]:
# fully connected model with 4 hidden layer

model = nn.Sequential(
        nn.Linear(1,6),
        nn.ReLU(),
        nn.Linear(6,10),
        nn.ReLU(),
        nn.Linear(10,6),
        nn.ReLU(),
        nn.Linear(6,1),
    ).cuda()

loss_func = nn.L1Loss()
optimizer = optim.SGD(model.parameters(),lr=0.0005)

## 4. Train 

In [11]:
# train
loss_arr = []
label = Variable(y_noise.cuda())

for i in range(num_epoch):
    optimizer.zero_grad()
    output = model(Variable(x.cuda()))

    loss = loss_func(output, label)
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        print(loss)

    loss_arr.append(loss.cpu().data.numpy())

tensor(78.0642, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(77.8424, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(77.5202, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(77.0138, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(76.0974, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(74.0172, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(67.1835, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(46.2326, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(32.9346, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(24.3279, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(23.8169, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(23.4013, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(22.9981, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(22.6016, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(22.2055, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(21.8072, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(21.4144, device='cuda:0', grad_fn

In [12]:
param_list = list(model.parameters())
print(param_list)

[Parameter containing:
tensor([[-0.8774],
        [ 0.4225],
        [-0.7410],
        [ 0.2316],
        [ 1.2405],
        [ 0.7290]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([ 0.2215, -1.9220, -1.7882, -0.9825,  0.2582, -0.6082], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([[ 0.2383,  0.1982, -0.2014,  0.2708, -0.0054, -0.3601],
        [-0.1966,  0.1701, -0.2341,  0.4209,  0.3554,  0.4279],
        [-0.3169,  0.1451, -0.0586,  0.1207, -0.0319, -0.3762],
        [ 0.1382,  0.3850,  0.7787, -0.0048, -0.0204,  0.2068],
        [-0.0103, -0.3111,  0.0435,  0.1310, -0.2576,  0.2257],
        [ 0.2719,  1.0341,  1.0175,  0.4127, -0.1697, -0.0342],
        [-0.4162,  0.9407,  1.3155,  0.5623, -0.1543,  0.5178],
        [ 0.1634,  0.2039, -0.2029, -0.4061, -0.0589,  0.2391],
        [ 0.3904,  0.7831,  0.4857,  0.2744,  0.0110,  0.3021],
        [-0.3413,  0.7361, -0.1232,  0.3679,  0.5168,  0.6336]],
       device='cuda:0', require

## 5. Visualize Trained Output

In [13]:
win_2 = vis.scatter(
    X = input_data,
    opts=dict(
        xtickmin=-15,
        xtickmax=15,
        xtickstep=1,
        ytickmin=0,
        ytickmax=500,
        ytickstep=1,
        markersymbol='dot',
        markercolor=np.random.randint(0, 255, num_data),
        markersize=5,
    ),
)

vis.line(
    X=x.reshape(x.shape[0]),     
    Y = output.cpu().data.reshape(output.cpu().data.shape[0]),
    win = win_2,    
)

'window_3a50ee51b67eb2'

## 6. Visualize Loss Graph

In [15]:
x = np.reshape([i for i in range(num_epoch)],newshape=[num_epoch,1])
loss_data = np.reshape(loss_arr,newshape=[num_epoch,1])

win3 = vis.line(
    X = x.reshape(x.shape[0]),
    Y = loss_data.reshape(loss_data.shape[0]),
)